In [ ]:
# import independent_functions as i_f ### THERE IS NO "independent_functions" IN THIS FOLDER, AND THERE SHOULDN'T BE
import pandas as pd
import numpy as np
import math
import scipy.stats as stats
from scipy.linalg import sqrtm
import scipy
import matplotlib.pyplot as plt
import time
import datetime
import random
import statistics

import numba
from numba import jit, njit
from numba.typed import List

# 

**הסבר על המחברת:**

היא בנויה מ3 חלקים, החלק הראשון להתעלם ממנו

החלק השני הוא חלק של וידוא שמירה על אלפא בסימולציות.

החלק השלישי הוא חלק של קבלת התוצאות (מספר הדחיות) מכל אחת מ5 הפרוצדורות.
עבור זה הייתי צריך לייצר פונקציות ספציפיות שרובן מסתיימות במילה 'סקולס'.

יש שם קושי להוציא מספר דחיות מהפרוצדורה של רות וסהרון פר בית ספר, כי בסוף הפרוצדורה שלהם דוחה אגרגטיבית. ואחר כך עוד קטע קוד קטן באותו היגיון על הפרוצדורה של 'סיאלאףדיאר' של דחיות פר בצפר.

ברגע שאני מסיים עם סיאלאףדיאר, אני מדפיס שם את מספר הדחיות בכל פרוצדורה, וחלקן היחסי מתוך גדלי בתי הספר.


#### START HERE:
- NEEDED:
    - 1.) 3 OMT alphas for certain 'iter_num's
    - 2.) all the functions involved, to be declared here


- This is built out of 2 parts: 
    - 1.) ensure alpha for the 3 OMT with mother that is dedicated to the 1 iter_num for the 3 OMT
    - 2.) create the line for the rejections of all the procedures (you may want, after, to maybe be able to distinguish between different school groups rejections...) 

# A.) Making sure that alpha is kept for the 3 OMT's 

In [ ]:
# 1.) setting of the J=5 groups

# relaxed nested list

# everything becomes a nested list of 3X5

num_hypo = [[347, 19, 2, 3, 53], # SMALL - FIRST 3 IS THE NULL
            [5440, 13, 7, 214, 224], # MEDIUM - FIRST 3 IS THE NULL
            [156, 475, 2, 41, 44]] # LARGE - FIRST 3 IS THE NULL

# i changed its name to "prob" because its really not "prob to one"
prob = [[0.818, 0.044, 0.004, 0.007 ,0.125], # SMALL - FIRST 3 IS THE NULL
        [0.922, 0.002, 0.001, 0.036, 0.037], # MEDIUM - FIRST 3 IS THE NULL
        [0.217, 0.661, 0.0027, 0.057, 0.061]] # LARGE - FIRST 3 IS THE NULL

mu0 = [[-0.061, 2.93, -13], # SMALL
      [-0.287, -7.13, 5.506], # MEDIUM 
      [4.575, -1.095, 43.3079]] # LARGE 

mu1 = [[11.867, -5.334], # SMALL 
      [-4.79, -22.392], # MEDIUM 
      [-28.613, -9.924]] # LARGE

# You should write here the REAL variance (sigma^2) [BTW, they're independent, not as in the DEPENDENT setting code]

variance_0 = [[1.24, 0.3936, 4.4625], # SMALL 
             [3.4692, 1538.411, 0.0117], # MEDIUM 
             [2.68, 5.2977, 14.5458]] # LARGE

variance_1 = [[2.0689, 2.6267], # SMALL 
             [26.5532, 21.7206], # MEDIUM 
             [20.5061, 6.6217]] # LARGE 

In [ ]:
# 2.) the optimal mus, found on a seperated process
musMargFDR_scalar = 471.21621
musMargPFDR_scalar = 467.65177
musMargMFDR_scalar = 0.57526

In [ ]:
iter_num = 200
alpha = 0.1

In [ ]:
def j5_fdep(alpha, num_hypo, prob_to_1, mu0, mu1, variance_0, variance_1, musMargFDR, musMargPFDR, musMargMFDR):
    
    #  columns order: marglocfdrFDR, marglocfdrpFDR, marglocfdrmFDR
    minprob_mat = np.zeros((1,6),float)
    lev_mat = np.zeros((1,6),float) 
    pow_mat = np.zeros((1,6),float) 
    ev_mat = np.zeros((1,6),float) 
    er_mat = np.zeros((1,6),float) 
    
    
    # RUTH AND SAHARON LOGIC IS MORE LIKE "CLFDR" - WE TAKE EVERYTHING FROM ALL THE GROUPS, AND THEN COMBINE
    block_beta = j5_my_rbeta(num_hypo, prob, mu0, mu1, variance_0, variance_1)
    locfdr = j5_locfdr_creator(block_beta, prob, mu0, mu1, variance_0, variance_1)
    
    marglocfdr_agg_stacked = np.hstack(locfdr)
    omarglocfdr = np.sort(marglocfdr_agg_stacked)
    amarg = 1 - omarglocfdr
    omarglocfdr_numba = numba.typed.List(omarglocfdr)    
    bmarg = np.array(BZCpp_numba_jit(omarglocfdr_numba)) # it WAS np.array but now it's a LIST of np.arrays so that when you take an index from it, it will be fine
    
    # marglocfdrFDR -v-     
    lev_mat, pow_mat, minprob_mat, ev_mat, er_mat = FDR_Generic_structure (mus=musMargFDR, a=amarg, b_1=bmarg, 
                                                                               b_2=bmarg, ind=3, 
                                                                               lev_mat=lev_mat, 
                                                                               pow_mat=pow_mat, 
                                                                               minprob_mat=minprob_mat, 
                                                                               ev_mat=ev_mat, 
                                                                               er_mat=er_mat,  
                                                                               olocfdr_function=omarglocfdr, 
                                                                               alpha = alpha)
    # marglocfdrpFDR -v-
    lev_mat, pow_mat, minprob_mat, ev_mat, er_mat = FDR_Generic_structure (mus=musMargPFDR, a=amarg, b_1=bmarg, 
                                                                               b_2=bmarg, ind=4, 
                                                                               lev_mat=lev_mat, 
                                                                               pow_mat=pow_mat, 
                                                                               minprob_mat=minprob_mat, 
                                                                               ev_mat=ev_mat, 
                                                                               er_mat=er_mat,  
                                                                               olocfdr_function=omarglocfdr,  
                                                                               alpha = alpha)
    # marglocfdrmFDR 
    lev_mat, pow_mat, minprob_mat, ev_mat, er_mat = FDR_Generic_structure (mus=musMargMFDR, a=amarg, b_1=(omarglocfdr - alpha),
                                                                               b_2=bmarg, ind=5, 
                                                                               lev_mat=lev_mat, 
                                                                               pow_mat=pow_mat, 
                                                                               minprob_mat=minprob_mat, 
                                                                               ev_mat=ev_mat, 
                                                                               er_mat=er_mat,  
                                                                               olocfdr_function=omarglocfdr, 
                                                                               alpha = alpha)
    return lev_mat, pow_mat, minprob_mat, ev_mat, er_mat

In [ ]:
# FINISHED - new locfdr building, per school size

# note: BEFORE FINISHING, I WASN'T SURE THAT THE BLOCK_BETA - MU / SIGMA IS THE RIGHT LOGIC, MEANING THE *WHOLE* VECTOR OF BLOCK_BETA...

def j5_locfdr_creator(block_beta, prob, mu0, mu1, variance_0, variance_1):

    locfdr_agg = []
    
    for i in range(len(prob)):
        
        # global prob_to_1
        prob_to_1 = np.sum(prob[i][3:])

        # standrad DISTRIBUTION
        dist_z = stats.norm(0, 1)
        #null
        standard_J1_null = (block_beta[i] - mu0[i][0])/np.sqrt(variance_0[0][0])
        standard_J2_null = (block_beta[i] - mu0[i][1])/np.sqrt(variance_0[0][1])
        standard_J3_null = (block_beta[i] - mu0[i][2])/np.sqrt(variance_0[0][2])
        #nonnull
        standard_J4_nonnull = (block_beta[i] - mu1[i][0])/np.sqrt(variance_1[0][0])
        standard_J5_nonnull = (block_beta[i] - mu1[i][1])/np.sqrt(variance_1[0][1])
        
        # components
        J1_gzh0 = ((prob[i][0])/(1-prob_to_1))*(1/np.sqrt(variance_0[i][0]))*dist_z.pdf(standard_J1_null)
        J2_gzh0 = ((prob[i][1])/(1-prob_to_1))*(1/np.sqrt(variance_0[i][1]))*dist_z.pdf(standard_J2_null)
        J3_gzh0 = ((prob[i][2])/(1-prob_to_1))*(1/np.sqrt(variance_0[i][2]))*dist_z.pdf(standard_J3_null) 
        J4_gzh1 = ((prob[i][3])/(prob_to_1))*(1/np.sqrt(variance_1[i][0]))*dist_z.pdf(standard_J4_nonnull)
        J5_gzh1 = ((prob[i][4])/(prob_to_1))*(1/np.sqrt(variance_1[i][1]))*dist_z.pdf(standard_J5_nonnull)

        gzh0 = J1_gzh0 + J2_gzh0 + J3_gzh0
        gzh1 = J4_gzh1 + J5_gzh1 

        locfdr_J5_marginal = ((1-prob_to_1)*gzh0) / (((1-prob_to_1)*gzh0)+(prob_to_1*gzh1))
        
        locfdr_agg.append(locfdr_J5_marginal)
    
    return locfdr_agg

In [ ]:
# FINISHED - new rbeta

def j5_my_rbeta(num_hypo, prob, mu0, mu1, variance_0, variance_1):
    
    block_beta_agg = []
    
    # For each school size
    for i in range(len(num_hypo)):
 
        # 5 J allocation
        multinomial_allocation = np.random.multinomial(sum(np.array(num_hypo[i])), prob[i], size=1)[0]

        # null
        J1_null = np.random.normal(mu0[i][0],np.sqrt(variance_0[i][0]),multinomial_allocation[0])
        J2_null = np.random.normal(mu0[i][1],np.sqrt(variance_0[i][1]),multinomial_allocation[1])
        J3_null = np.random.normal(mu0[i][2],np.sqrt(variance_0[i][2]),multinomial_allocation[2])

        #nonnull
        J4_nonnull = np.random.normal(mu1[i][0],np.sqrt(variance_1[i][0]),multinomial_allocation[3])
        J5_nonnull = np.random.normal(mu1[i][1],np.sqrt(variance_1[i][1]),multinomial_allocation[4])

        # and concat them together 
        J_args = (J1_null, J2_null, J3_null, J4_nonnull, J5_nonnull)
        block_beta = np.concatenate(J_args)
        
        # append
        block_beta_agg.append(block_beta)
        
    return block_beta_agg

In [ ]:
def func_marg_FDR_j5(mu_FDR_variable, iter_num, alpha, num_hypo, prob_to_1, mu0, mu1, variance_0, variance_1, musMargPFDR, musMargMFDR):
    start = time.time()
    lev_mat_agg = pow_mat_agg = minprob_mat_agg = ev_mat_agg = er_mat_agg = 0
    for i in range(iter_num):
            
        lev_mat, pow_mat, minprob_mat, ev_mat, er_mat = j5_fdep(alpha, num_hypo, prob_to_1, mu0, mu1, variance_0, variance_1, 
                                                                                [mu_FDR_variable], 
                                                                                musMargPFDR, musMargMFDR)
        lev_mat_agg += lev_mat[0]
        pow_mat_agg += pow_mat[0]
        minprob_mat_agg += minprob_mat[0]
        ev_mat_agg += ev_mat[0]
        er_mat_agg += er_mat[0]
        
    # R & S RETURN
    pow_mat_r = pow_mat_agg / iter_num
    lev_mat_r = lev_mat_agg / iter_num
    ev_mat_r = ev_mat_agg / iter_num
    er_mat_r = er_mat_agg / iter_num
    minprob_mat_r = minprob_mat_agg / iter_num
    
    return pow_mat_r, lev_mat_r, ev_mat_r, er_mat_r, minprob_mat_r

In [ ]:
musMargFDR_scalar = [471.21621]
musMargPFDR_scalar = [467.65177]
musMargMFDR_scalar = [0.57526]

In [ ]:
a, b, c, d, e = func_marg_FDR_j5(musMargFDR_scalar, iter_num, alpha, num_hypo, prob, mu0, mu1, variance_0, variance_1, musMargPFDR_scalar, musMargMFDR_scalar)

In [ ]:
a, b, c, d, e 

In [ ]:
### THIS IS THE ACCEPTANCE FOR KEEPING 0.1=ALPHA

b

In [ ]:
def mother_procedure_schools_NOCS(iter_num, alpha, num_hypo, prob_to_1, mu0, mu1, variance_0, variance_1, musMargFDR_scalar, musMargPFDR_scalar, musMargMFDR_scalar):
    
    start = time.time()    
    
    # r & s
    #rs_power, rs_fdr, rs_mfdr_V, rs_mfdr_R, rs_pfdr_minprob = [], [], [], [], []
    lev_mat_agg = pow_mat_agg = minprob_mat_agg = ev_mat_agg = er_mat_agg = 0
    
    
    for i in range(iter_num):
         

        # our BLOCK BETA needs to be created each iteration, as far as i understand
        
        block_beta_schools_simu = j5_my_rbeta(num_hypo, prob_to_1, mu0, mu1, variance_0, variance_1)

        # R&S WHAT WE HAVE IN SCHOOLS NOW
        lev_mat, pow_mat, minprob_mat, ev_mat, er_mat = fdep_marginals_groups_schools(block_beta_schools_simu, alpha, num_hypo, prob_to_1, mu0, mu1, variance_0, variance_1, musMargFDR_scalar, musMargPFDR_scalar, musMargMFDR_scalar)
        
        ## r & s
        #lev_mat, pow_mat, minprob_mat, ev_mat, er_mat = fdep_marginals_groups (alpha, num_hypo, prob_to_1, mu0, mu1, variance_0, variance_1, musMargFDR_scalar, musMargPFDR_scalar, musMargMFDR_scalar)
        
        lev_mat_agg += lev_mat[0]
        pow_mat_agg += pow_mat[0]
        minprob_mat_agg += minprob_mat[0]
        ev_mat_agg += ev_mat[0]
        er_mat_agg += er_mat[0]
        
        
        
    # R & S
    # (IF I UNDERSTAND CORRECTRLY, WITH R & S WE WILL HAVE 3 X 4 = 12 NEW METRICS: EACH OF THE 4 METRIC FOR EACH Err Optimizer)
    pow_mat_r = pow_mat_agg / iter_num
    lev_mat_r = lev_mat_agg / iter_num
    ev_mat_r = ev_mat_agg / iter_num
    er_mat_r = er_mat_agg / iter_num
    minprob_mat_r = minprob_mat_agg / iter_num
    
    ## marginallocfdr FDR control
    rs_margFDR_power_r = pow_mat_r[3]
    rs_margFDR_fdr_r = lev_mat_r[3]
    rs_margFDR_mfdr_r = ev_mat_r[3] / er_mat_r[3]
    rs_margFDR_pfdr_r = lev_mat_r[3] / minprob_mat_r[3]

    ## marginallocfdr pFDR control
    rs_margpFDR_power_r = pow_mat_r[4]
    rs_margpFDR_fdr_r = lev_mat_r[4]
    rs_margpFDR_mfdr_r = ev_mat_r[4] / er_mat_r[4]
    rs_margpFDR_pfdr_r = lev_mat_r[4] / minprob_mat_r[4]
    
    ## marginallocfdr mFDR control
    rs_margmFDR_power_r = pow_mat_r[5]
    rs_margmFDR_fdr_r = lev_mat_r[5]
    rs_margmFDR_mfdr_r = ev_mat_r[5] / er_mat_r[5]
    rs_margmFDR_pfdr_r = lev_mat_r[5] / minprob_mat_r[5]
    
    # FNR for each one of the 5 procedures   
    rs_margFDR_fnr_r = FNR(num_hypo, prob_to_1, er_mat_r[3], ev_mat_r[3]) 
    rs_margpFDR_fnr_r = FNR(num_hypo, prob_to_1, er_mat_r[4], ev_mat_r[4])
    rs_margmFDR_fnr_r = FNR(num_hypo, prob_to_1, er_mat_r[5], ev_mat_r[5])
    
    # Arranging the final output
    rs_margFDR_list = [rs_margFDR_power_r, rs_margFDR_fdr_r, rs_margFDR_mfdr_r, rs_margFDR_pfdr_r, rs_margFDR_fnr_r]
    rs_margpFDR_list = [rs_margpFDR_power_r, rs_margpFDR_fdr_r, rs_margpFDR_mfdr_r, rs_margpFDR_pfdr_r, rs_margpFDR_fnr_r]
    rs_margmFDR_list = [rs_margmFDR_power_r, rs_margmFDR_fdr_r, rs_margmFDR_mfdr_r, rs_margmFDR_pfdr_r, rs_margmFDR_fnr_r]

    stop = time.time()
    duration = stop-start
    print("TIME IT TOOK FOR THE REST OF MOTHER TO RUN, AFTER ALL THE ITERATIONS WERE DONE: " + str(duration))
    return rs_margFDR_list, rs_margpFDR_list, rs_margmFDR_list


In [ ]:
# THE mother, that will check if alpha is surely kept

slfdr_list, clfdr_list, rs_margFDR_list, rs_margpFDR_list, rs_margmFDR_list = mother_procedure_schools_NOCS(5, alpha, num_hypo, prob, mu0, mu1, variance_0, variance_1, musMargFDR_scalar, musMargPFDR_scalar, musMargMFDR_scalar)

slfdr_list, clfdr_list, rs_margFDR_list, rs_margpFDR_list, rs_margmFDR_list

A needed function

In [ ]:
# this ONE, and the other down with its dependencies

In [ ]:
# FINISHED - new rbeta

def j5_my_rbeta(num_hypo, prob, mu0, mu1, variance_0, variance_1):
    
    block_beta_agg = []
    
    # For each school size
    for i in range(len(num_hypo)):
 
        # 5 J allocation
        multinomial_allocation = np.random.multinomial(sum(np.array(num_hypo[i])), prob[i], size=1)[0]

        # null
        J1_null = np.random.normal(mu0[i][0],np.sqrt(variance_0[i][0]),multinomial_allocation[0])
        J2_null = np.random.normal(mu0[i][1],np.sqrt(variance_0[i][1]),multinomial_allocation[1])
        J3_null = np.random.normal(mu0[i][2],np.sqrt(variance_0[i][2]),multinomial_allocation[2])

        #nonnull
        J4_nonnull = np.random.normal(mu1[i][0],np.sqrt(variance_1[i][0]),multinomial_allocation[3])
        J5_nonnull = np.random.normal(mu1[i][1],np.sqrt(variance_1[i][1]),multinomial_allocation[4])

        # and concat them together 
        J_args = (J1_null, J2_null, J3_null, J4_nonnull, J5_nonnull)
        block_beta = np.concatenate(J_args)
        
        # append
        block_beta_agg.append(block_beta)
        
    return block_beta_agg

# B.) The Data Analysis Results

- Final results for the thesis
- Input is the 3 different schools vectors (block_beta_schools) - output is the power and rejections
- Input for fdep is the 3 optimal Mu's as well

#### Building the REJECTION TOOL for 1 realization

**(RUN THIS TO INITIATE THE FUNCTIONS)**

In [ ]:
# 1.) Bring all "mother_procedure" dependencies
# 2.) Adapt them to a 1 realization solution

In [ ]:
### 0.) fdep ONLY

### NEW - TAKING THE LOCFDR FROM THE OUTSIDE - - - 9.7.2022 I DON'T KNOW WHY I WROTE 'LOCFDR', I MIGHT HAVE MENT 'BLOCK BETA'

def fdep_marginals_groups_schools(block_beta_schools, alpha, num_hypo, prob_to_1, mu0, mu1, variance_0, variance_1, musMargFDR, musMargPFDR, musMargMFDR):

    """ block_beta_schools is a list of 3 schools z vectors (instead of "block_beta" X 3 times) """
    marglocfdr_agg = []
    
    #  columns order: marglocfdrFDR, marglocfdrpFDR, marglocfdrmFDR
    minprob_mat = np.zeros((1,6),float)
    lev_mat = np.zeros((1,6),float) 
    pow_mat = np.zeros((1,6),float) 
    ev_mat = np.zeros((1,6),float) 
    er_mat = np.zeros((1,6),float) 
    
    
    # RUTH AND SAHARON LOGIC IS MORE LIKE "CLFDR" - WE TAKE EVERYTHING FROM ALL THE GROUPS, AND THEN COMBINE
    for i in range(len(mu0)):
        # - - - BLOCK BETA - - - is now the Z vector (list of 3 vectors) : "block_beta_schools"

        # - - - for marginal rule - - -
        dist_0 = stats.norm(mu0[i], np.sqrt(variance_0[i]))
        dist_1 = stats.norm(mu1[i], np.sqrt(variance_1[i]))
        margprob_ = (1-prob_to_1[i]) * dist_0.pdf(block_beta_schools[i])  +  prob_to_1[i] * dist_1.pdf(block_beta_schools[i])
        marglocfdr = (1-prob_to_1[i]) * dist_0.pdf(block_beta_schools[i]) / margprob_    
        marglocfdr_agg.append(marglocfdr)
    
    marglocfdr_agg_stacked = np.hstack(marglocfdr_agg)
    omarglocfdr = np.sort(marglocfdr_agg_stacked)
    amarg = 1 - omarglocfdr
    omarglocfdr_numba = numba.typed.List(omarglocfdr)    
    bmarg = np.array(BZCpp_numba_jit(omarglocfdr_numba)) # it WAS np.array but now it's a LIST of np.arrays so that when you take an index from it, it will be fine
    
    # marglocfdrFDR -v-     
    lev_mat, pow_mat, minprob_mat, ev_mat, er_mat = FDR_Generic_structure (mus=musMargFDR, a=amarg, b_1=bmarg, 
                                                                               b_2=bmarg, ind=3, 
                                                                               lev_mat=lev_mat, 
                                                                               pow_mat=pow_mat, 
                                                                               minprob_mat=minprob_mat, 
                                                                               ev_mat=ev_mat, 
                                                                               er_mat=er_mat,  
                                                                               olocfdr_function=omarglocfdr, 
                                                                               alpha = alpha)
    # marglocfdrpFDR -v-
    lev_mat, pow_mat, minprob_mat, ev_mat, er_mat = FDR_Generic_structure (mus=musMargPFDR, a=amarg, b_1=bmarg, 
                                                                               b_2=bmarg, ind=4, 
                                                                               lev_mat=lev_mat, 
                                                                               pow_mat=pow_mat, 
                                                                               minprob_mat=minprob_mat, 
                                                                               ev_mat=ev_mat, 
                                                                               er_mat=er_mat,  
                                                                               olocfdr_function=omarglocfdr,  
                                                                               alpha = alpha)
    # marglocfdrmFDR 
    lev_mat, pow_mat, minprob_mat, ev_mat, er_mat = FDR_Generic_structure (mus=musMargMFDR, a=amarg, b_1=(omarglocfdr - alpha),
                                                                               b_2=bmarg, ind=5, 
                                                                               lev_mat=lev_mat, 
                                                                               pow_mat=pow_mat, 
                                                                               minprob_mat=minprob_mat, 
                                                                               ev_mat=ev_mat, 
                                                                               er_mat=er_mat,  
                                                                               olocfdr_function=omarglocfdr, 
                                                                               alpha = alpha)
    return lev_mat, pow_mat, minprob_mat, ev_mat, er_mat



In [ ]:

## 2.3.) Ruth & Saharon

def DCpp_numba(R):
    K=len(R)
    D=[0]*K
    cR=0

    i=0
    while i<K:
        cR=0
        j=i
        while j<K:
            cR+=R[j]
            if cR>0:
                D[i]=1
                j+=1    # I think that this one is useless
                break
            j+=1
        if D[i]==0:
            i+=1
            break    
        i+=1
    return D
DCpp_numba_jit = jit()(DCpp_numba)


def BZCpp_numba(Tz):
    K= len(Tz)
    S = np.zeros((K,K))
    b = [0.0]*K
    
    S[0,0] = 1
    i = 1
    while i<K:
        S[i,0] = S[i-1,0]*(1-Tz[i-1])
        v=1
        while v<=i:
            S[i,v] = S[i-1,v]*(1-Tz[i-1]) + S[i-1,v-1]*Tz[i-1]
            v+=1
        i+=1
    b[0] = S[0,0]*Tz[0]
    i = 1
    while i<K:
        j = 0
        while j<=i:
            b[i] += S[i,j]*(((i-j)*Tz[i] - j*(1-Tz[i]))/(i*(i+1)))
            j+=1
        i+=1
    return b 
BZCpp_numba_jit = jit()(BZCpp_numba)


def FDR_Generic_structure (mus, a, b_1, b_2, ind, lev_mat, pow_mat, minprob_mat, ev_mat, er_mat, olocfdr_function, alpha = 0.05):
    for mui in range(len(mus)):                ##### we use "mui" as an INDEX
        mu = mus[mui]
        Rz = a-mu*b_1
        if (ind == 1) | (ind == 4):
            Rz[0] = Rz[0] + mu*alpha
            
        Rz_numba = numba.typed.List(Rz)
        Dz = list(DCpp_numba_jit(Rz_numba))
        
        indices_ab = [i for i, x in enumerate(Dz) if x == 1]
        sum_b = sum([b_2[j] for j in indices_ab])
        ### if sum_b <0: # it's here because u had a problem with that
        lev_mat[mui, ind] = lev_mat[mui,ind] + sum_b 
        sum_a = sum([a[j] for j in indices_ab])  
        pow_mat[mui, ind] = pow_mat[mui,ind] + sum_a 
        minprob_mat[mui, ind] = minprob_mat[mui,ind] + Dz[0] 
        sum_oloc = sum([olocfdr_function[j] for j in indices_ab])
        ev_mat[mui, ind] = ev_mat[mui, ind] + sum_oloc 
        er_mat[mui, ind] = er_mat[mui, ind] + sum(Dz) 
    return lev_mat, pow_mat, minprob_mat, ev_mat, er_mat

In [ ]:
### ONLY SLFDR_RULE

def SLFDR_decision_rule(locfdr, alpha):
    # recieves: SORTED np.array - locfdr of some group "k"  +  alpha
    # returns: number of rejections + olocfdr of the rejected ones
    
    locfdr_cumsum = np.cumsum(locfdr)
    rule_sums = locfdr_cumsum / np.array(list(range(1, len(locfdr_cumsum)+1))) 
    
    for statistic in rule_sums[::-1]:
        if statistic <= alpha:
            break
            
    if (statistic == rule_sums[0]) & (statistic > alpha):        
        num_rejections = 0
        rejections_olocfdr = 0        
        
    else:
        num_rejections = list(rule_sums).index(statistic) + 1
        rejections_olocfdr = locfdr[:num_rejections]
        
    return num_rejections, rejections_olocfdr

In [ ]:

### NEW

## 2.1.) SLFDR - - - - WHY THERE'S final_V????

def my_SLFDR_schools (block_beta_schools, alpha, num_hypo, prob_to_1, mu0, mu1, variance_0, variance_1): 
    
    """ block_beta_schools is a list of 3 schools z vectors (instead of "block_beta" X 3 times) """
    
    locfdr_agg = []
    final_R = [] 
    final_V = []
    
    for i in range(len(mu0)):    
        ###################################################################### creating RBETA instance 
        ### no need anymore
        
        ###################################################################### creating marginal_locfdr 
        dist_0 = stats.norm(mu0[i], np.sqrt(variance_0[i]))
        dist_1 = stats.norm(mu1[i], np.sqrt(variance_1[i]))
        margprob_ = (1-prob_to_1[i]) * dist_0.pdf(block_beta_schools[i])  +  prob_to_1[i] * dist_1.pdf(block_beta_schools[i])
        marglocfdr = (1-prob_to_1[i]) * dist_0.pdf(block_beta_schools[i]) / margprob_                        
        omarglocfdr = np.sort(marglocfdr)
  
        ###################################################################### Decision rule
        num_rejections, rejections_olocfdr = SLFDR_decision_rule(omarglocfdr, alpha)
        
        final_R.append(num_rejections)
        locfdr_agg.append(rejections_olocfdr)
    
    return locfdr_agg ,final_R


### NEW

## 2.2.) CLFDR - - - - WHY THERE'S 'VEC_H'????

def my_CLFDR_schools (block_beta_schools, alpha, num_hypo, prob_to_1, mu0, mu1, variance_0, variance_1): 
    vec_h_agg = []
    locfdr_agg = []
    
    for i in range(len(mu0)):    
        ###################################################################### creating RBETA instance 
        ### no need anymore
        
        ###################################################################### creating marginal_locfdr 
        dist_0 = stats.norm(mu0[i], np.sqrt(variance_0[i]))
        dist_1 = stats.norm(mu1[i], np.sqrt(variance_1[i]))
        margprob_ = (1-prob_to_1[i]) * dist_0.pdf(block_beta_schools[i])  +  prob_to_1[i] * dist_1.pdf(block_beta_schools[i])
        marglocfdr = (1-prob_to_1[i]) * dist_0.pdf(block_beta_schools[i]) / margprob_                        
  
        ### Add it to the overall lists
        vec_h_agg.append(vec_h)
        locfdr_agg.append(marglocfdr)

    ###################################################################### h_stacking & Decision rule - using the SLFDR decision rule for the aggregated locfdrs
    locfdr_agg_stacked = np.hstack(locfdr_agg)
    omarglocfdr = np.sort(locfdr_agg_stacked)
    num_rejections, rejections_olocfdr = SLFDR_decision_rule(omarglocfdr, alpha)

    return rejections_olocfdr, num_rejections 


In [ ]:
# z_small = pd.read_csv('z_small.txt', header = None)
# z_medium = pd.read_csv('z_medium.txt', header = None)
# z_large = pd.read_csv('z_large.txt', header = None)

In [ ]:
z_small = pd.read_csv('z_small.txt', header = None)
z_medium = pd.read_csv('z_medium.txt', header = None)
z_large = pd.read_csv('z_large.txt', header = None)

block_beta_schools = [np.array(z_small[0]), np.array(z_medium[0]), np.array(z_large[0])]

In [ ]:
z_small = pd.read_csv('z_small.txt', header = None)
z_medium = pd.read_csv('z_medium.txt', header = None)
z_large = pd.read_csv('z_large.txt', header = None)

block_beta_schools = [np.array(z_small[0]), np.array(z_medium[0]), np.array(z_large[0])]

musMargFDR_scalar = [471.21621]
musMargPFDR_scalar = [467.65177]
musMargMFDR_scalar = [0.57526]

In [ ]:

def mother_procedure_schools (block_beta_schools, alpha, num_hypo, prob_to_1, mu0, mu1, variance_0, variance_1, musMargFDR_scalar, musMargPFDR_scalar, musMargMFDR_scalar):
    
    start = time.time()    

    # SLFDR
    locfdr_agg, R_sl = my_SLFDR_schools (block_beta_schools, alpha, num_hypo, prob_to_1, mu0, mu1, variance_0, variance_1)

    # CLFDR
    rejections_olocfdr ,R_cl = my_CLFDR_schools (block_beta_schools, alpha, num_hypo, prob_to_1, mu0, mu1, variance_0, variance_1)

    # r & s
    lev_mat, pow_mat, minprob_mat, ev_mat, er_mat = fdep_marginals_groups_schools(block_beta_schools, alpha, num_hypo, prob_to_1, mu0, mu1, variance_0, variance_1, musMargFDR_scalar, musMargPFDR_scalar, musMargMFDR_scalar)

    stop = time.time()
    duration = stop-start
    print("TIME IT TOOK FOR THE REST OF MOTHER TO RUN, AFTER ALL THE ITERATIONS WERE DONE: " + str(duration))
    
    return  R_sl, R_cl, er_mat_r[3], er_mat_r[4], er_mat_r[5]


In [ ]:

a,b,c,d,e =  mother_procedure_schools (block_beta_schools, alpha, num_hypo, prob, mu0, mu1, variance_0, variance_1, musMargFDR_scalar, musMargPFDR_scalar, musMargMFDR_scalar)

In [ ]:

# def j5_locfdr_creator(block_beta, prob, mu0, mu1, variance_0, variance_1):

#     locfdr_agg = []
    
#     for i in range(len(prob)):
        
#         # global prob_to_1
#         prob_to_1 = np.sum(prob[i][3:])

#         # standrad DISTRIBUTION
#         dist_z = stats.norm(0, 1)
#         #null
#         standard_J1_null = (block_beta[i] - mu0[i][0])/np.sqrt(variance_0[0][0])
#         standard_J2_null = (block_beta[i] - mu0[i][1])/np.sqrt(variance_0[0][1])
#         standard_J3_null = (block_beta[i] - mu0[i][2])/np.sqrt(variance_0[0][2])
#         #nonnull
#         standard_J4_nonnull = (block_beta[i] - mu1[i][0])/np.sqrt(variance_1[0][0])
#         standard_J5_nonnull = (block_beta[i] - mu1[i][1])/np.sqrt(variance_1[0][1])
        
#         # components
#         J1_gzh0 = ((prob[i][0])/(1-prob_to_1))*(1/np.sqrt(variance_0[i][0]))*dist_z.pdf(standard_J1_null)
#         J2_gzh0 = ((prob[i][1])/(1-prob_to_1))*(1/np.sqrt(variance_0[i][1]))*dist_z.pdf(standard_J2_null)
#         J3_gzh0 = ((prob[i][2])/(1-prob_to_1))*(1/np.sqrt(variance_0[i][2]))*dist_z.pdf(standard_J3_null) 
#         J4_gzh1 = ((prob[i][3])/(prob_to_1))*(1/np.sqrt(variance_1[i][0]))*dist_z.pdf(standard_J4_nonnull)
#         J5_gzh1 = ((prob[i][4])/(prob_to_1))*(1/np.sqrt(variance_1[i][1]))*dist_z.pdf(standard_J5_nonnull)

#         gzh0 = J1_gzh0 + J2_gzh0 + J3_gzh0
#         gzh1 = J4_gzh1 + J5_gzh1 

#         locfdr_J5_marginal = ((1-prob_to_1)*gzh0) / (((1-prob_to_1)*gzh0)+(prob_to_1*gzh1))
        
#         locfdr_agg.append(locfdr_J5_marginal)
    
#     return locfdr_agg

#### Building the new locfdr logic for the data analysis functions

In [ ]:

def j5_locfdr_creator_marginal_group(i, block_beta, prob, mu0, mu1, variance_0, variance_1):

    # global prob_to_1
    prob_to_1 = np.sum(prob[i][3:])

    # standrad DISTRIBUTION
    dist_z = stats.norm(0, 1)
    #null
    standard_J1_null = (block_beta[i] - mu0[i][0])/np.sqrt(variance_0[0][0])
    standard_J2_null = (block_beta[i] - mu0[i][1])/np.sqrt(variance_0[0][1])
    standard_J3_null = (block_beta[i] - mu0[i][2])/np.sqrt(variance_0[0][2])
    #nonnull
    standard_J4_nonnull = (block_beta[i] - mu1[i][0])/np.sqrt(variance_1[0][0])
    standard_J5_nonnull = (block_beta[i] - mu1[i][1])/np.sqrt(variance_1[0][1])
        
    # components
    J1_gzh0 = ((prob[i][0])/(1-prob_to_1))*(1/np.sqrt(variance_0[i][0]))*dist_z.pdf(standard_J1_null)
    J2_gzh0 = ((prob[i][1])/(1-prob_to_1))*(1/np.sqrt(variance_0[i][1]))*dist_z.pdf(standard_J2_null)
    J3_gzh0 = ((prob[i][2])/(1-prob_to_1))*(1/np.sqrt(variance_0[i][2]))*dist_z.pdf(standard_J3_null) 
    J4_gzh1 = ((prob[i][3])/(prob_to_1))*(1/np.sqrt(variance_1[i][0]))*dist_z.pdf(standard_J4_nonnull)
    J5_gzh1 = ((prob[i][4])/(prob_to_1))*(1/np.sqrt(variance_1[i][1]))*dist_z.pdf(standard_J5_nonnull)

    gzh0 = J1_gzh0 + J2_gzh0 + J3_gzh0
    gzh1 = J4_gzh1 + J5_gzh1 

    locfdr_J5_marginal = ((1-prob_to_1)*gzh0) / (((1-prob_to_1)*gzh0)+(prob_to_1*gzh1))
    
    return locfdr_J5_marginal

In [ ]:
### ONLY SLFDR_RULE

def SLFDR_decision_rule(locfdr, alpha):
    # recieves: SORTED np.array - locfdr of some group "k"  +  alpha
    # returns: number of rejections + olocfdr of the rejected ones
    
    locfdr_cumsum = np.cumsum(locfdr)
    rule_sums = locfdr_cumsum / np.array(list(range(1, len(locfdr_cumsum)+1))) 
    
    for statistic in rule_sums[::-1]:
        if statistic <= alpha:
            break
            
    if (statistic == rule_sums[0]) & (statistic > alpha):        
        num_rejections = 0
        rejections_olocfdr = 0        
        
    else:
        num_rejections = list(rule_sums).index(statistic) + 1
        rejections_olocfdr = locfdr[:num_rejections]
        
    return num_rejections, rejections_olocfdr

In [ ]:
## SLFDR 

def my_SLFDR_schools (block_beta_schools, alpha, num_hypo, prob_to_1, mu0, mu1, variance_0, variance_1): 
    
    """ block_beta_schools is a list of 3 schools z vectors (instead of "block_beta" X 3 times) """
    
    locfdr_agg = []
    final_R = [] 
    final_V = []
    
    for i in range(len(mu0)):    
        ###################################################################### creating RBETA instance 
        ### no need anymore
        
        ###################################################################### creating marginal_locfdr 
        #dist_0 = stats.norm(mu0[i], np.sqrt(variance_0[i]))
        #dist_1 = stats.norm(mu1[i], np.sqrt(variance_1[i]))
        #margprob_ = (1-prob_to_1[i]) * dist_0.pdf(block_beta_schools[i])  +  prob_to_1[i] * dist_1.pdf(block_beta_schools[i])
        #marglocfdr = (1-prob_to_1[i]) * dist_0.pdf(block_beta_schools[i]) / margprob_    
        
        marglocfdr = j5_locfdr_creator_marginal_group(i, block_beta_schools, prob, mu0, mu1, variance_0, variance_1)
        
        omarglocfdr = np.sort(marglocfdr)
  
        ###################################################################### Decision rule
        num_rejections, rejections_olocfdr = SLFDR_decision_rule(omarglocfdr, alpha)
        
        final_R.append(num_rejections)
        locfdr_agg.append(rejections_olocfdr)
    
    return locfdr_agg ,final_R


## CLFDR 

def my_CLFDR_schools (block_beta_schools, alpha, num_hypo, prob_to_1, mu0, mu1, variance_0, variance_1): 
    locfdr_agg = []
    
    for i in range(len(mu0)):    
        ###################################################################### creating RBETA instance 
        ### no need anymore
        
        ###################################################################### creating marginal_locfdr 
        #dist_0 = stats.norm(mu0[i], np.sqrt(variance_0[i]))
        #dist_1 = stats.norm(mu1[i], np.sqrt(variance_1[i]))
        #margprob_ = (1-prob_to_1[i]) * dist_0.pdf(block_beta_schools[i])  +  prob_to_1[i] * dist_1.pdf(block_beta_schools[i])
        #marglocfdr = (1-prob_to_1[i]) * dist_0.pdf(block_beta_schools[i]) / margprob_                        
        
        marglocfdr = j5_locfdr_creator_marginal_group(i, block_beta_schools, prob, mu0, mu1, variance_0, variance_1)        
        
        ### Add it to the overall lists
        locfdr_agg.append(marglocfdr)

    ###################################################################### h_stacking & Decision rule - using the SLFDR decision rule for the aggregated locfdrs

    locfdr_agg_stacked = np.hstack(locfdr_agg)
    omarglocfdr = np.sort(locfdr_agg_stacked)
    num_rejections, rejections_olocfdr = SLFDR_decision_rule(omarglocfdr, alpha)

    return rejections_olocfdr, num_rejections 


In [ ]:

## 2.3.) Ruth & Saharon

def DCpp_numba(R):
    K=len(R)
    D=[0]*K
    cR=0

    i=0
    while i<K:
        cR=0
        j=i
        while j<K:
            cR+=R[j]
            if cR>0:
                D[i]=1
                j+=1    # I think that this one is useless
                break
            j+=1
        if D[i]==0:
            i+=1
            break    
        i+=1
    return D
DCpp_numba_jit = jit()(DCpp_numba)


def BZCpp_numba(Tz):
    K= len(Tz)
    S = np.zeros((K,K))
    b = [0.0]*K
    
    S[0,0] = 1
    i = 1
    while i<K:
        S[i,0] = S[i-1,0]*(1-Tz[i-1])
        v=1
        while v<=i:
            S[i,v] = S[i-1,v]*(1-Tz[i-1]) + S[i-1,v-1]*Tz[i-1]
            v+=1
        i+=1
    b[0] = S[0,0]*Tz[0]
    i = 1
    while i<K:
        j = 0
        while j<=i:
            b[i] += S[i,j]*(((i-j)*Tz[i] - j*(1-Tz[i]))/(i*(i+1)))
            j+=1
        i+=1
    return b 
BZCpp_numba_jit = jit()(BZCpp_numba)


def FDR_Generic_structure (mus, a, b_1, b_2, ind, lev_mat, pow_mat, minprob_mat, ev_mat, er_mat, olocfdr_function, alpha = 0.05):
    for mui in range(len(mus)):                ##### we use "mui" as an INDEX
        mu = mus[mui]
        Rz = a-mu*b_1
        if (ind == 1) | (ind == 4):
            Rz[0] = Rz[0] + mu*alpha
            
        Rz_numba = numba.typed.List(Rz)
        Dz = list(DCpp_numba_jit(Rz_numba))
        
        indices_ab = [i for i, x in enumerate(Dz) if x == 1]
        sum_b = sum([b_2[j] for j in indices_ab])
        ### if sum_b <0: # it's here because u had a problem with that
        lev_mat[mui, ind] = lev_mat[mui,ind] + sum_b 
        sum_a = sum([a[j] for j in indices_ab])  
        pow_mat[mui, ind] = pow_mat[mui,ind] + sum_a 
        minprob_mat[mui, ind] = minprob_mat[mui,ind] + Dz[0] 
        sum_oloc = sum([olocfdr_function[j] for j in indices_ab])
        ev_mat[mui, ind] = ev_mat[mui, ind] + sum_oloc 
        er_mat[mui, ind] = er_mat[mui, ind] + sum(Dz) 
    return lev_mat, pow_mat, minprob_mat, ev_mat, er_mat

In [ ]:
### 0.) fdep ONLY

### NEW - TAKING THE LOCFDR FROM THE OUTSIDE - - - 9.7.2022 I DON'T KNOW WHY I WROTE 'LOCFDR', I MIGHT HAVE MENT 'BLOCK BETA'

def fdep_marginals_groups_schools(block_beta_schools, alpha, num_hypo, prob_to_1, mu0, mu1, variance_0, variance_1, musMargFDR, musMargPFDR, musMargMFDR):

    """ block_beta_schools is a list of 3 schools z vectors (instead of "block_beta" X 3 times) """
    marglocfdr_agg = []
    
    #  columns order: marglocfdrFDR, marglocfdrpFDR, marglocfdrmFDR
    minprob_mat = np.zeros((1,6),float)
    lev_mat = np.zeros((1,6),float) 
    pow_mat = np.zeros((1,6),float) 
    ev_mat = np.zeros((1,6),float) 
    er_mat = np.zeros((1,6),float) 
    
    
    # RUTH AND SAHARON LOGIC IS MORE LIKE "CLFDR" - WE TAKE EVERYTHING FROM ALL THE GROUPS, AND THEN COMBINE
    for i in range(len(mu0)):
        # - - - BLOCK BETA - - - is now the Z vector (list of 3 vectors) : "block_beta_schools"

        # - - - for marginal rule - - -
        #dist_0 = stats.norm(mu0[i], np.sqrt(variance_0[i]))
        #dist_1 = stats.norm(mu1[i], np.sqrt(variance_1[i]))
        #margprob_ = (1-prob_to_1[i]) * dist_0.pdf(block_beta_schools[i])  +  prob_to_1[i] * dist_1.pdf(block_beta_schools[i])
        #marglocfdr = (1-prob_to_1[i]) * dist_0.pdf(block_beta_schools[i]) / margprob_  
        
        marglocfdr = j5_locfdr_creator_marginal_group(i, block_beta_schools, prob, mu0, mu1, variance_0, variance_1)        

        marglocfdr_agg.append(marglocfdr)

    marglocfdr_agg_stacked = np.hstack(marglocfdr_agg)
    omarglocfdr = np.sort(marglocfdr_agg_stacked)
    amarg = 1 - omarglocfdr
    omarglocfdr_numba = numba.typed.List(omarglocfdr)    
    bmarg = np.array(BZCpp_numba_jit(omarglocfdr_numba)) # it WAS np.array but now it's a LIST of np.arrays so that when you take an index from it, it will be fine
    
    # marglocfdrFDR -v-     
    lev_mat, pow_mat, minprob_mat, ev_mat, er_mat = FDR_Generic_structure (mus=musMargFDR, a=amarg, b_1=bmarg, 
                                                                               b_2=bmarg, ind=3, 
                                                                               lev_mat=lev_mat, 
                                                                               pow_mat=pow_mat, 
                                                                               minprob_mat=minprob_mat, 
                                                                               ev_mat=ev_mat, 
                                                                               er_mat=er_mat,  
                                                                               olocfdr_function=omarglocfdr, 
                                                                               alpha = alpha)
    # marglocfdrpFDR -v-
    lev_mat, pow_mat, minprob_mat, ev_mat, er_mat = FDR_Generic_structure (mus=musMargPFDR, a=amarg, b_1=bmarg, 
                                                                               b_2=bmarg, ind=4, 
                                                                               lev_mat=lev_mat, 
                                                                               pow_mat=pow_mat, 
                                                                               minprob_mat=minprob_mat, 
                                                                               ev_mat=ev_mat, 
                                                                               er_mat=er_mat,  
                                                                               olocfdr_function=omarglocfdr,  
                                                                               alpha = alpha)
    # marglocfdrmFDR 
    lev_mat, pow_mat, minprob_mat, ev_mat, er_mat = FDR_Generic_structure (mus=musMargMFDR, a=amarg, b_1=(omarglocfdr - alpha),
                                                                               b_2=bmarg, ind=5, 
                                                                               lev_mat=lev_mat, 
                                                                               pow_mat=pow_mat, 
                                                                               minprob_mat=minprob_mat, 
                                                                               ev_mat=ev_mat, 
                                                                               er_mat=er_mat,  
                                                                               olocfdr_function=omarglocfdr, 
                                                                               alpha = alpha)
    #print(er_mat[0][3], er_mat[0][4], er_mat[0][5])
    rej_fdr_s, rej_fdr_m, rej_fdr_l = rej_per_school(marglocfdr_agg_stacked, int(er_mat[0][3]))
    rej_fdr = [rej_fdr_s, rej_fdr_m, rej_fdr_l]
    rej_pfdr_s, rej_pfdr_m, rej_pfdr_l = rej_per_school(marglocfdr_agg_stacked, int(er_mat[0][4]))
    rej_pfdr = [rej_pfdr_s, rej_pfdr_m, rej_pfdr_l]
    rej_mfdr_s, rej_mfdr_m, rej_mfdr_l = rej_per_school(marglocfdr_agg_stacked, int(er_mat[0][5]))
    rej_mfdr = [rej_mfdr_s, rej_mfdr_m, rej_mfdr_l]
    
    return lev_mat, pow_mat, minprob_mat, ev_mat, er_mat, rej_fdr, rej_pfdr, rej_mfdr



In [ ]:
z_small = pd.read_csv('z_small.txt', header = None)
z_medium = pd.read_csv('z_medium.txt', header = None)
z_large = pd.read_csv('z_large.txt', header = None)

block_beta_schools = [np.array(z_small[0]), np.array(z_medium[0]), np.array(z_large[0])]

#block_beta_schools = [list(z_small[0]), list(z_medium[0]), list(z_large[0])]


musMargFDR_scalar = [471.21621]
musMargPFDR_scalar = [467.65177]
musMargMFDR_scalar = [0.57526]


In [ ]:
def mother_procedure_schools (block_beta_schools, alpha, num_hypo, prob_to_1, mu0, mu1, variance_0, variance_1, musMargFDR_scalar, musMargPFDR_scalar, musMargMFDR_scalar):
    
    start = time.time()    

    # SLFDR
    locfdr_agg, R_sl = my_SLFDR_schools (block_beta_schools, alpha, num_hypo, prob_to_1, mu0, mu1, variance_0, variance_1)

    # CLFDR
    rejections_olocfdr ,R_cl, rej_clfdr = my_CLFDR_schools (block_beta_schools, alpha, num_hypo, prob_to_1, mu0, mu1, variance_0, variance_1)

    # r & s
    lev_mat, pow_mat, minprob_mat, ev_mat, er_mat, rej_fdr, rej_pfdr, rej_mfdr = fdep_marginals_groups_schools(block_beta_schools, alpha, num_hypo, prob_to_1, mu0, mu1, variance_0, variance_1, musMargFDR_scalar, musMargPFDR_scalar, musMargMFDR_scalar)

    stop = time.time()
    duration = stop-start
    #print("TIME IT TOOK FOR THE REST OF MOTHER TO RUN, AFTER ALL THE ITERATIONS WERE DONE: " + str(duration))
    #print(er_mat)
    return  R_sl, R_cl, rej_clfdr, er_mat[0][3], er_mat[0][4], er_mat[0][5], rej_fdr, rej_pfdr, rej_mfdr


In [ ]:
a,b,c,d,e =  mother_procedure_schools (block_beta_schools, alpha, num_hypo, prob, mu0, mu1, variance_0, variance_1, musMargFDR_scalar, musMargPFDR_scalar, musMargMFDR_scalar)

In [ ]:
a ,b,c,d,e

In [ ]:
np.sum(np.array(a))

In [ ]:
len(block_beta_schools[0]) + len(block_beta_schools[1]) + len(block_beta_schools[2])

In [ ]:
len(block_beta_schools[0])/7040 , len(block_beta_schools[1])/7040 , len(block_beta_schools[2])/7040

In [ ]:
a[0]/np.sum(np.array(a)) #, a[1]/np.sum(np.array(a), a[2]/np.sum(np.array(a))

In [ ]:
a[1]/np.sum(np.array(a))

In [ ]:
a[2] / np.sum(np.array(a))

In [ ]:
len(block_beta_schools[0]) / len(block_beta_schools[1])

### Identifyinng rejected schools from within groups, for the OMTs

In [ ]:
def FDR_Generic_structure (mus, a, b_1, b_2, ind, lev_mat, pow_mat, minprob_mat, ev_mat, er_mat, olocfdr_function, alpha = 0.05):
    for mui in range(len(mus)):                ##### we use "mui" as an INDEX
        mu = mus[mui]
        Rz = a-mu*b_1
        #print(Rz)
        if (ind == 1) | (ind == 4):
            Rz[0] = Rz[0] + mu*alpha
            
        Rz_numba = numba.typed.List(Rz)
        Dz = list(DCpp_numba_jit(Rz_numba))
        #print(len(Dz)) # it comes 3 times because it's running for 3 procedures
        #print(Dz[0])
        #print(len(Dz[1]))
        #print(len(Dz[2]))
        indices_ab = [i for i, x in enumerate(Dz) if x == 1]
        sum_b = sum([b_2[j] for j in indices_ab])
        ### if sum_b <0: # it's here because u had a problem with that
        lev_mat[mui, ind] = lev_mat[mui,ind] + sum_b 
        sum_a = sum([a[j] for j in indices_ab])  
        pow_mat[mui, ind] = pow_mat[mui,ind] + sum_a 
        minprob_mat[mui, ind] = minprob_mat[mui,ind] + Dz[0] 
        sum_oloc = sum([olocfdr_function[j] for j in indices_ab])
        ev_mat[mui, ind] = ev_mat[mui, ind] + sum_oloc 
        er_mat[mui, ind] = er_mat[mui, ind] + sum(Dz) 
    return lev_mat, pow_mat, minprob_mat, ev_mat, er_mat

In [ ]:
a,b,c,d,e =  mother_procedure_schools (block_beta_schools, alpha, num_hypo, prob, mu0, mu1, variance_0, variance_1, musMargFDR_scalar, musMargPFDR_scalar, musMargMFDR_scalar)
a,b,c,d,e 

In [ ]:
### THIS IS THE EXAMPLE FOR YOU

## 2.1.) SLFDR
def my_SLFDR (alpha, num_hypo, prob_to_1, mu0, mu1, variance_0, variance_1): 
    locfdr_agg = []
    final_R = [] 
    final_V = []
    
    for i in range(len(mu0)):    
        ###################################################################### creating RBETA instance 
        block_beta, vec_h = my_rbeta(num_hypo[i], 
                                     prob_to_1[i], 
                                     mu0[i], 
                                     mu1[i], 
                                     variance_0[i], 
                                     variance_1[i],
                                     True)
        ###################################################################### creating marginal_locfdr 
        dist_0 = stats.norm(mu0[i], np.sqrt(variance_0[i]))
        dist_1 = stats.norm(mu1[i], np.sqrt(variance_1[i]))
        margprob_ = (1-prob_to_1[i]) * dist_0.pdf(block_beta)  +  prob_to_1[i] * dist_1.pdf(block_beta)
        marglocfdr = (1-prob_to_1[i]) * dist_0.pdf(block_beta) / margprob_                        
        omarglocfdr = np.sort(marglocfdr)
  
        ###################################################################### Decision rule
        num_rejections, rejections_olocfdr = SLFDR_decision_rule(omarglocfdr, alpha)
        
        ###################################################################### CREATING "V" (for each group):
        
        #1. take the "marglocfdr" & "vec_h" , enumerate it as a dictionary 
        marglocfdr_d = dict(enumerate(marglocfdr))
        vec_h_d = dict(enumerate(vec_h))
        
        #2. sort the locfdr (values), so that their index (keys) is shuffeled
        omarglocfdr_d = dict(sorted(marglocfdr_d.items(), key=lambda x: x[1]))
    
        #3. take only the "num_rejections" first SHUFFELED indexes from "omarglocfdr_d"
        first_indexes = list(omarglocfdr_d.keys())[:num_rejections]

        #4. take only the "first_indexes" indexes from the enumerated vec_h dictionary, the vector of their values: "values_h"
        final_h = dict([(k, vec_h_d[k]) for k in first_indexes])

        #5. V = num_rejections - sum(values_h)
        V = num_rejections - sum(final_h.values())
        
        final_R.append(num_rejections)
        locfdr_agg.append(rejections_olocfdr)
        final_V.append(V)
    
    minprob = 0
    if sum(final_R) > 0:
        minprob = 1

    return locfdr_agg ,final_R, final_V, minprob 


In [ ]:
small_sc = np.array([0.01,0.002,0.5,0.22])
medium_sc = np.array([0.01,0.002,0.5,0.22])

In [ ]:
dict(enumerate(small_sc))

In [ ]:
{hi:j for j in np.array(3,4,5)}

In [ ]:
len(block_beta_schools[0]), len(block_beta_schools[1]), len(block_beta_schools[2])

In [ ]:
# inputs: marglocfdr_agg_stacked, er_mat[0][3][4][5]


#1. take the "marglocfdr_agg_stacked", enumerate it as a dictionary while knowing: small: 424, medium: 5898, large: 718
marglocfdr_agg_stacked_d = dict(enumerate(marglocfdr_agg_stacked))

#2. sort the locfdr (values), their index (keys) is shuffeled
omarglocfdr_agg_stacked_d = dict(sorted(marglocfdr_agg_stacked_d.items(), key=lambda x: x[1]))

#3. take only the "er_mat[0] - - - [3][4][5]" first SHUFFELED indexes from "omarglocfdr_agg_stacked_d"
first_indexes = list(omarglocfdr_agg_stacked_d.keys())[:er_mat[0][3]]

# now COUNT the indexes from 1-424 (real: 0-423), 424-6322 (real: 424-6321), 6322-end (7040, real: 6322-7039)
small_rej = 0
medium_rej = 0
large_rej = 0

for i in range(len(first_indexes)):
    if first_indexes < 424:
        small_rej+=1
    elif first_indexes < 6322:
        medium_rej+=1
    else:
        large_rej+=1

In [ ]:
5898+424+718

In [ ]:
small_sc = np.array([0.01,0.002,0.5,0.22])

marglocfdr_agg_stacked_d = dict(enumerate(small_sc))

list(marglocfdr_agg_stacked_d.keys())[:2]

#### schools function in R&S

In [ ]:
def rej_per_school(marglocfdr_agg_stacked, er_mat):
    
    #1. take the "marglocfdr_agg_stacked", enumerate it as a dictionary while knowing: small: 424, medium: 5898, large: 718
    marglocfdr_agg_stacked_d = dict(enumerate(marglocfdr_agg_stacked))
    
    #2. sort the locfdr (values), their index (keys) is shuffeled
    omarglocfdr_agg_stacked_d = dict(sorted(marglocfdr_agg_stacked_d.items(), key=lambda x: x[1]))
    
    #3. take only the "er_mat[0] - - - [3][4][5]" first SHUFFELED indexes from "omarglocfdr_agg_stacked_d"
    first_indexes = list(omarglocfdr_agg_stacked_d.keys())[:er_mat]
    
    #4. now COUNT the indexes from 1-424 (real: 0-423), 424-6322 (real: 424-6321), 6322-end (7040, real: 6322-7039)
    small_rej = 0
    medium_rej = 0
    large_rej = 0
        
    for i in range(len(first_indexes)):
        if first_indexes[i] < 424:
            small_rej+=1
        if first_indexes[i] < 6322:
            medium_rej+=1
        else:
            large_rej+=1
    
    return small_rej, medium_rej, large_rej


In [ ]:
# first run after function
a,b,c,d,e, h, i, j =  mother_procedure_schools (block_beta_schools, alpha, num_hypo, prob, mu0, mu1, variance_0, variance_1, musMargFDR_scalar, musMargPFDR_scalar, musMargMFDR_scalar)

In [ ]:
a,b,c,d,e, h, i, j

#### CLFDR  

In [ ]:
## NEW & GOOD SLFDR


## CLFDR 

def my_CLFDR_schools (block_beta_schools, alpha, num_hypo, prob_to_1, mu0, mu1, variance_0, variance_1): 
    locfdr_agg = []
    
    for i in range(len(mu0)):    
        ###################################################################### creating RBETA instance 
        ### no need anymore
        
        ###################################################################### creating marginal_locfdr 
        #dist_0 = stats.norm(mu0[i], np.sqrt(variance_0[i]))
        #dist_1 = stats.norm(mu1[i], np.sqrt(variance_1[i]))
        #margprob_ = (1-prob_to_1[i]) * dist_0.pdf(block_beta_schools[i])  +  prob_to_1[i] * dist_1.pdf(block_beta_schools[i])
        #marglocfdr = (1-prob_to_1[i]) * dist_0.pdf(block_beta_schools[i]) / margprob_                        
        
        marglocfdr = j5_locfdr_creator_marginal_group(i, block_beta_schools, prob, mu0, mu1, variance_0, variance_1)        
        
        ### Add it to the overall lists
        locfdr_agg.append(marglocfdr)

    ###################################################################### h_stacking & Decision rule - using the SLFDR decision rule for the aggregated locfdrs
    locfdr_agg_stacked = np.hstack(locfdr_agg)
    omarglocfdr = np.sort(locfdr_agg_stacked)
    num_rejections, rejections_olocfdr = SLFDR_decision_rule(omarglocfdr, alpha)

    return rejections_olocfdr, num_rejections 




    marglocfdr_agg_stacked = np.hstack(marglocfdr_agg)






## OLD & GOOD - 2.2.) CLFDR
def my_CLFDR (alpha, num_hypo, prob_to_1, mu0, mu1, variance_0, variance_1): 
    vec_h_agg = []
    locfdr_agg = []
    
    for i in range(len(mu0)):    
        ###################################################################### creating RBETA instance 
        block_beta, vec_h = my_rbeta(num_hypo[i], 
                                     prob_to_1[i], 
                                     mu0[i], 
                                     mu1[i], 
                                     variance_0[i], 
                                     variance_1[i],
                                     True)
        ###################################################################### creating marginal_locfdr 
        dist_0 = stats.norm(mu0[i], np.sqrt(variance_0[i]))
        dist_1 = stats.norm(mu1[i], np.sqrt(variance_1[i]))
        margprob_ = (1-prob_to_1[i]) * dist_0.pdf(block_beta)  +  prob_to_1[i] * dist_1.pdf(block_beta)
        marglocfdr = (1-prob_to_1[i]) * dist_0.pdf(block_beta) / margprob_                        
  
        ### Add it to the overall lists
        vec_h_agg.append(vec_h)
        locfdr_agg.append(marglocfdr)

    ###################################################################### h_stacking & Decision rule - using the SLFDR decision rule for the aggregated locfdrs
    vec_h_agg_stacked = np.hstack(vec_h_agg)
    locfdr_agg_stacked = np.hstack(locfdr_agg)
    omarglocfdr = np.sort(locfdr_agg_stacked)
    
    num_rejections, rejections_olocfdr = SLFDR_decision_rule(omarglocfdr, alpha)
        
    ###################################################################### CREATING "V" (for each group):
        
    #1. take the "marglocfdr" & "vec_h" , enumerate it as a dictionary 
    marglocfdr_d = dict(enumerate(locfdr_agg_stacked))
    vec_h_d = dict(enumerate(vec_h_agg_stacked))
        
    #2. sort the locfdr (values), so that their index (keys) is shuffeled
    omarglocfdr_d = dict(sorted(marglocfdr_d.items(), key=lambda x: x[1]))
    
    #3. take only the "num_rejections" first SHUFFELED indexes from "omarglocfdr_d"
    first_indexes = list(omarglocfdr_d.keys())[:num_rejections]

    #4. take only the "first_indexes" indexes from the enumerated vec_h dictionary, the vector of their values: "values_h"
    final_h = dict([(k, vec_h_d[k]) for k in first_indexes])

    #5. V = num_rejections - sum(values_h)
    V = num_rejections - sum(final_h.values())
    
    # adding minprob (=1 if R>0, =0 else)
    minprob = 0
    if num_rejections > 0:
        minprob = 1

    # return: 1.) locfdrs  2.) final_R  3.) final_V  4.) minprob
    return rejections_olocfdr, num_rejections, V, minprob 


In [ ]:
# first run after function
a,b,c,d,e, h, i, j =  mother_procedure_schools (block_beta_schools, alpha, num_hypo, prob, mu0, mu1, variance_0, variance_1, musMargFDR_scalar, musMargPFDR_scalar, musMargMFDR_scalar)
a,b,c,d,e, h, i, j

#### per schools for CLFDR 

In [ ]:
# NEW CLFDR SCHOOLS

## CLFDR 

def my_CLFDR_schools (block_beta_schools, alpha, num_hypo, prob_to_1, mu0, mu1, variance_0, variance_1): 
    locfdr_agg = []
    
    for i in range(len(mu0)):    
        ###################################################################### creating RBETA instance 
        ### no need anymore
        
        ###################################################################### creating marginal_locfdr 
        #dist_0 = stats.norm(mu0[i], np.sqrt(variance_0[i]))
        #dist_1 = stats.norm(mu1[i], np.sqrt(variance_1[i]))
        #margprob_ = (1-prob_to_1[i]) * dist_0.pdf(block_beta_schools[i])  +  prob_to_1[i] * dist_1.pdf(block_beta_schools[i])
        #marglocfdr = (1-prob_to_1[i]) * dist_0.pdf(block_beta_schools[i]) / margprob_                        
        
        marglocfdr = j5_locfdr_creator_marginal_group(i, block_beta_schools, prob, mu0, mu1, variance_0, variance_1)        
        
        ### Add it to the overall lists
        locfdr_agg.append(marglocfdr)

    ###################################################################### h_stacking & Decision rule - using the SLFDR decision rule for the aggregated locfdrs

    locfdr_agg_stacked = np.hstack(locfdr_agg)
    omarglocfdr = np.sort(locfdr_agg_stacked)
    num_rejections, rejections_olocfdr = SLFDR_decision_rule(omarglocfdr, alpha)

    rej_clfdr_s, rej_clfdr_m, rej_clfdr_l = rej_per_school(locfdr_agg_stacked, num_rejections)
    rej_clfdr = [rej_clfdr_s, rej_clfdr_m, rej_clfdr_l]
    
    return rejections_olocfdr, num_rejections, rej_clfdr 


In [ ]:
# first run after function
a,b,c,d,e, h, i, j, k =  mother_procedure_schools (block_beta_schools, alpha, num_hypo, prob, mu0, mu1, variance_0, variance_1, musMargFDR_scalar, musMargPFDR_scalar, musMargMFDR_scalar)
a,b,c,d,e, h, i, j, k

In [ ]:
len(block_beta_schools[0])/7040 , len(block_beta_schools[1])/7040 , len(block_beta_schools[2])/7040

# """original fractions"""
